In [1]:
import torch
from torch import nn

In [118]:
num_actor_obs = 45 * 100
actor_hidden_dims = [64, 256, 128]
num_actions = 12
activation = nn.ELU()
kernel_sizes = [5, 5, 5, 5, 5]
strides = [2, 2, 2, 2, 2]
filters = [32, 32, 32, 32, 32]
paddings = [2, 2, 2, 2, 2]
dilations = [1, 1, 1, 1, 1]

out_channels = filters[:]
in_channels = [1] + filters[:-1]

In [119]:
obs = torch.rand(128, num_actor_obs)
obs.shape

torch.Size([128, 4500])

In [120]:
actor_layers = []
mlp_input_dim_a = num_actor_obs
for in_ch, out_ch, kernel_size, stride, padding, dilation in zip(in_channels, out_channels, kernel_sizes, strides, paddings, dilations):
    actor_layers.append(nn.Conv1d(
        in_channels=in_ch,
        out_channels=out_ch,
        kernel_size=kernel_size,
        stride=stride,
        padding=padding,
        dilation=dilation
    ))
    actor_layers.append(activation)
    print((mlp_input_dim_a + 2 * padding - dilation * (kernel_size - 1) - 1) // stride + 1, ((mlp_input_dim_a + 2 * padding - dilation * (kernel_size - 1) - 1) // stride + 1) * out_ch)
    print(((mlp_input_dim_a + 2 * padding - dilation * (kernel_size - 1) - 1) // stride + 1) * stride >= mlp_input_dim_a) 
    mlp_input_dim_a = (mlp_input_dim_a + 2 * padding - dilation * (kernel_size - 1) - 1) // stride + 1

actor_layers.append(nn.Flatten())
mlp_input_dim_a = mlp_input_dim_a * out_channels[-1]
print(mlp_input_dim_a)

actor_layers.append(nn.Linear(mlp_input_dim_a, actor_hidden_dims[0]))
actor_layers.append(activation)
for layer_index in range(len(actor_hidden_dims)):
    if layer_index == len(actor_hidden_dims) - 1:
        actor_layers.append(nn.Linear(actor_hidden_dims[layer_index], num_actions))
    else:
        actor_layers.append(nn.Linear(actor_hidden_dims[layer_index], actor_hidden_dims[layer_index + 1]))
        actor_layers.append(activation)
actor = nn.Sequential(*actor_layers)

2250 72000
True
1125 36000
True
563 18016
True
282 9024
True
141 4512
True
4512


In [121]:
sum([p.numel() for p in actor.parameters()])

360716

In [151]:
num_actor_obs_h = 10
num_actor_obs_w = 45

actor_hidden_dims = [64, 256, 128]
num_actions = 12
activation = nn.ELU()
kernel_sizes = [(5, 5), (5, 5), (5, 5), (5, 5)]
strides = [(2, 2), (2, 2), (2, 2), (2, 2)]
filters = [32, 32, 32, 32]
paddings = [(2, 2), (2, 2), (2, 2), (2, 2)]
dilations = [(1, 1), (1, 1), (1, 1), (1, 1)]

out_channels = filters[:]
in_channels = [1] + filters[:-1]

In [156]:
obs = torch.rand(10, num_actor_obs_h, num_actor_obs_w)

In [ ]:
actor_layers = []
out_w = num_actor_obs_w
out_h = num_actor_obs_h

for in_ch, out_ch, kernel_size, stride, padding, dilation in zip(in_channels, out_channels, kernel_sizes, strides, paddings, dilations):
    actor_layers.append(nn.Conv2d(
        in_channels=in_ch,
        out_channels=out_ch,
        kernel_size=kernel_size,
        stride=stride,
        padding=padding,
        dilation=dilation
    ))
    actor_layers.append(activation)
    last_h = out_h
    last_w = out_w
    out_h = (out_h + 2 * padding[0] - dilation[0] * (kernel_size[0] - 1) - 1) // stride[0] + 1
    out_w = (out_w + 2 * padding[1] - dilation[1] * (kernel_size[1] - 1) - 1) // stride[1] + 1

    print(f'out_h: {out_h} out_w: {out_w} total: {out_h * out_w * out_ch}')
    print(f'pad_h: {out_h * stride[0], out_h * stride[0] >= last_h} pad_w: {out_w * stride[1], out_w * stride[1] >= last_w}')

actor_layers.append(nn.Flatten())
mlp_input_dim_a = out_h * out_w * out_channels[-1]

print(mlp_input_dim_a)

actor_layers.append(nn.Linear(mlp_input_dim_a, actor_hidden_dims[0]))
actor_layers.append(activation)
for layer_index in range(len(actor_hidden_dims)):
    if layer_index == len(actor_hidden_dims) - 1:
        actor_layers.append(nn.Linear(actor_hidden_dims[layer_index], num_actions))
    else:
        actor_layers.append(nn.Linear(actor_hidden_dims[layer_index], actor_hidden_dims[layer_index + 1]))
        actor_layers.append(activation)
actor = nn.Sequential(*actor_layers)

out_h: 5 out_w: 23 total: 3680
pad_h: (10, True) pad_w: True
out_h: 3 out_w: 12 total: 1152
pad_h: (6, True) pad_w: True
out_h: 2 out_w: 6 total: 384
pad_h: (4, True) pad_w: True
out_h: 1 out_w: 3 total: 96
pad_h: (2, True) pad_w: True
96


In [158]:
sum([p.numel() for p in actor.parameters()])

135020

In [162]:
actor(obs.unsqueeze(1)).shape

torch.Size([10, 12])